In [2]:
import cv2
import os
import json
import glob

In [3]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [4]:
def load_json_values(directory_path):
    # 디렉토리 안의 모든 JSON 파일 경로를 가져오기
    json_files = glob.glob(f"{directory_path}/*.json")
    # JSON 파일들의 값만을 저장할 리스트
    json_values = []

    # 각 JSON 파일의 값을 읽어서 리스트에 추가
    for file_path in json_files:
        with open(file_path, 'r') as file:
            try:
                data = json.load(file)
                json_values.append(data)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {file_path}: {e}")

    return json_values


In [5]:
def get_mp4_titles(directory_path):
    # 디렉토리 안의 모든 MP4 파일 경로를 가져오기
    mp4_files = glob.glob(f"{directory_path}/*.mp4")

    # MP4 파일들의 제목을 저장할 리스트
    mp4_titles = []

    # 각 MP4 파일의 제목을 추출하여 리스트에 추가
    for mp4_file in mp4_files:
        _, file_title = os.path.split(mp4_file)
        file_title, _ = os.path.splitext(file_title)  # 파일명과 확장자 분리
        mp4_titles.append(file_title)  # 각 제목을 리스트에 담아 추가

    return mp4_titles

In [6]:
# 특정 디렉토리 경로를 지정
JSON_path = 'C:/Users/sunxo/Study/BIS/data/JSON/흡연' # 조정 값 0
MP4_path = 'C:/Users/sunxo/Study/BIS/data/movie/흡연' # 조정 값 1

# JSON 파일들을 불러오기
loaded_json_data = load_json_values(JSON_path)

# MP4 파일들의 제목 불러오기
mp4_titles = get_mp4_titles(MP4_path)

# MP4 경로 수정
base_path = 'C:/Users/sunxo/Study/BIS/data/movie/흡연/' # 조정 값 2
mp4_ospath= []
mp4_ospath = [base_path + i for i in mp4_titles]

In [7]:
# JSON 파일에서 frame 구하기

count = 0
frame_list = []

for i in range(len(loaded_json_data)):
    count = len(loaded_json_data[i])
    temp_list = []  

    for j in range(count):
        temp = loaded_json_data[i][j].get('frame')
        temp_list.append(temp)

    frame_list.append(temp_list)


In [8]:
# 이미지 파일 저장소 생성

directory_paths = []

for i in range(len(mp4_ospath)):
    name = os.path.basename(mp4_ospath[i])
    try:
        directory_path = "C:/Users/sunxo/Study/BIS/data/images/smoke/" + name                       # 조정 값 3
        directory_paths.append(directory_path)  # 리스트에 추가
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)
            print(f"Directory created successfully. Path: {directory_path}")
        else:
            print("Directory already exists. Path:", directory_path)
    except OSError as e:
        print(f"Error: Creating directory. {str(e)}")


Directory created successfully. Path: C:/Users/sunxo/Study/BIS/data/images/smoke/C_3_10_10_BU_DYA_08-04_11-53-22_CA_RGB_DF2_F2
Directory created successfully. Path: C:/Users/sunxo/Study/BIS/data/images/smoke/C_3_10_10_BU_DYA_08-04_11-53-22_CB_RGB_DF2_F2
Directory created successfully. Path: C:/Users/sunxo/Study/BIS/data/images/smoke/C_3_10_10_BU_DYA_08-04_11-53-22_CC_RGB_DF2_F2
Directory created successfully. Path: C:/Users/sunxo/Study/BIS/data/images/smoke/C_3_10_10_BU_DYB_10-17_14-47-30_CB_RGB_DF2_F2
Directory created successfully. Path: C:/Users/sunxo/Study/BIS/data/images/smoke/C_3_10_10_BU_SMA_09-20_14-28-59_CA_RGB_DF2_M2
Directory created successfully. Path: C:/Users/sunxo/Study/BIS/data/images/smoke/C_3_10_10_BU_SMA_09-20_14-28-59_CB_RGB_DF2_M2
Directory created successfully. Path: C:/Users/sunxo/Study/BIS/data/images/smoke/C_3_10_10_BU_SMA_09-20_14-28-59_CC_RGB_DF2_M2
Directory created successfully. Path: C:/Users/sunxo/Study/BIS/data/images/smoke/C_3_10_10_BU_SMA_09-20_14-28-5

In [9]:
mp4_ospath[0]

'C:/Users/sunxo/Study/BIS/data/movie/흡연/C_3_10_10_BU_DYA_08-04_11-53-22_CA_RGB_DF2_F2'

In [10]:
mp4_filename = [path + '.mp4' for path in mp4_ospath]

for i in range(len(mp4_ospath)):
    filepath = mp4_filename[i]

    if not os.path.exists(filepath):
        print(filepath)
        continue

    try:
        video = cv2.VideoCapture(filepath)
        if not video.isOpened():
            raise Exception("비디오 파일을 열 수 없습니다.")

        count = 0  # 각 파일마다 count 초기화

        while video.isOpened():
            ret, image = video.read()

            if not ret:
                break

            # 현재 프레임 번호 확인
            current_frame = str(int(video.get(1)))
                
            if current_frame in frame_list[i]:
                # 이미지 저장
                filename = os.path.join(directory_paths[i], f"{os.path.basename(filepath)[:-4]}_{frame_list[i][count]}.jpg")
                # 파일 이름에 있는 역슬래시를 슬래시로 변경
                filename = filename.replace("\\", "/")
                
                cv2.imwrite(filename, image)

                print(f'Saved frame number: {frame_list[i][count]}, filename: {filename}')
                
                # 카운터 증가
                count += 1

    except Exception as e:
        print(f"오류 발생 ({filepath}): {str(e)}")
        
        continue

    finally:
        video.release()
